In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
import numpy as np

In [2]:
from tensorflow.keras.layers import Layer
class DenseResidual(Layer):
    def __init__(self, units=32):
        super().__init__()
        self.dense = tf.keras.layers.Dense(units)
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.add = tf.keras.layers.Add()
    def call(self, inputs):
        x = inputs
        for i in range(2):
            x = self.dense(x)
        x = self.batch_norm(x)
        
        return self.add([inputs, x])

In [3]:
class ConvResidual(Layer):
    def __init__(self, filters=64, kernal_size=3, activation=None):
        super().__init__()
        self.conv2d = tf.keras.layers.Conv2D(filters, kernal_size, padding='same', activation=activation)
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.add = tf.keras.layers.Add()
    def call(self, inputs):
        x = inputs
        for i in range(2):
            x = self.conv2d(x)
        x = self.batch_norm(x)
        return self.add([inputs, x])

In [4]:
class Resnet(tf.keras.Model):
    def __init__(self, num_of_classes=10):
        super(Resnet, self).__init__()
        self.input1 = tf.keras.layers.InputLayer(input_shape=(28, 28,1))
        self.dense1 = tf.keras.layers.Dense(32)
        self.convresidual = ConvResidual(32, 3, 'relu')
        self.denseresidual = DenseResidual(32)
        self.global_pool = tf.keras.layers.GlobalAveragePooling2D()
        self.dense2 = tf.keras.layers.Dense(num_of_classes, activation="softmax")
    def call(self, inputs, training=False):
        x = self.input1(inputs)
        x = self.dense1(x)
        x = self.convresidual(x)
        for i in range(3):
            x = self.denseresidual(x)
        x = self.global_pool(x)
        x = self.dense2(x)
        return x

In [5]:
resnet = Resnet(10)


In [7]:
# p = resnet(np.expand_dims(tf.zeros([1, 2, 3]),0))
# resnet.summary()

Model: "resnet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
dense (Dense)                multiple                  128       
_________________________________________________________________
conv_residual (ConvResidual) multiple                  9376      
_________________________________________________________________
dense_residual (DenseResidua multiple                  1184      
_________________________________________________________________
global_average_pooling2d (Gl multiple                  0         
_________________________________________________________________
dense_2 (Dense)              multiple                  330       
Total params: 11,018
Trainable params: 10,890
Non-trainable params: 128
______________________________________________________

In [6]:
resnet.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [7]:
(x_train, y_train), (x_test, y_test)= mnist.load_data()
x_train = x_train.astype('float32')/255.0
x_test = x_test.astype('float32')/255.0
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')


In [8]:
resnet.fit(np.expand_dims(np.array(x_train),-1), np.array(y_train), 
                validation_data=(np.expand_dims(np.array(x_test),-1), np.array(y_test)), 
                epochs=2, batch_size=32)

Train on 60000 samples, validate on 10000 samples
Epoch 1/2
60000/60000 [==============================] - 302s 5ms/sample - loss: 0.5352 - accuracy: 0.8503 - val_loss: 0.8142 - val_accuracy: 0.7181
Epoch 2/2
60000/60000 [==============================] - 332s 6ms/sample - loss: 0.1992 - accuracy: 0.9403 - val_loss: 0.8265 - val_accuracy: 0.7035
